# Create Slice for Fabric Rack
This is the slice creation for the ```Exploring Queues``` tutorial, This is the first step of the assignment, to create the slice on fabric using fablib
 

## 1. Set up the Experiment

In this section you will use the Fablib maager to create a new slice that is composed of 3 nodes for the Exploring Queues Assignment.
### 1.1 Reserve Resources
In this section we will declare the Structure of the Slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "Queues"
site = "EDUKY"

image = "default_ubuntu_20"
nicmodel = "NIC_Basic"

cores = 1
ram = 2
disk = 10

try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    #Router
    router = slice.add_node(name="router", site=site)
    router.set_capacities(cores=cores, ram=ram, disk=disk)
    router.set_image(image)
    rPort1 = router.add_component(model=nicmodel, name="rPort1").get_interfaces()[0] 
    rPort2 = router.add_component(model=nicmodel, name="rPort2").get_interfaces()[0] 
    
    #Host 1
    host1 = slice.add_node(name="Source", site=site)
    host1.set_capacities(cores=cores, ram=ram, disk=disk)
    host1.set_image(image)
    h1Port = host1.add_component(model=nicmodel, name="h1Port").get_interfaces()[0] 
    
    #Host 2
    host2 = slice.add_node(name="Destination", site=site)
    host2.set_capacities(cores=cores, ram=ram, disk=disk)
    host2.set_image(image)
    h2Port = host2.add_component(model=nicmodel, name="h2Port").get_interfaces()[0] 
    
    lan1 = slice.add_l2network(name="Lan1", interfaces=[rPort1, h1Port])
    lan2 = slice.add_l2network(name="Lan2", interfaces=[rPort2, h2Port])
    
    #Submit Slice Request
    slice.submit()
    
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
This section of the slice creation, once the nodes are active, will asign the network spaces some predefined values that we will use for the Assgnment

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    host1 = slice.get_node(name="Source") 
    host2 = slice.get_node(name="Destination")
    router = slice.get_node(name="router")
    
    subnet1 = IPv4Network("10.1.1.0/24")
    subnet2 = IPv4Network("11.1.1.0/24")
    
    host1_iface = host1.get_interface(network_name="Lan1")
    host1_iface.ip_addr_add(addr="10.1.1.1", subnet=subnet1)
    
    router_iface = router.get_interface(network_name="Lan1")
    router_iface.ip_addr_add(addr="10.1.1.2", subnet=subnet1)
    
    router_iface2 = router.get_interface(network_name="Lan2")
    router_iface2.ip_addr_add(addr="11.1.1.2", subnet=subnet2)
    
    host2_iface2 = host2.get_interface(network_name="Lan2")
    host2_iface2.ip_addr_add(addr="11.1.1.1", subnet=subnet2) 
    
    #turn on interfaces
    for node in slice.get_nodes():
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
    
    host1.execute("sudo ip route add 11.1.1.0/24 via 10.1.1.2")
    router.execute("sudo sysctl -w net.ipv4.ip_forward=1")
    host2.execute("sudo ip route add 10.1.1.0/24 via 11.1.1.2")
except Exception as e:
    print(f"Exception: {e}")


### 1.3 Configure the software needed for the nodes in the experiment
This section of the slice creation, we will Add any aditional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
for node in slice.get_nodes():
    node.execute("sudo apt update;sudo apt -y install iperf")

## Continue to The experiment notebook

Once You have completed this notebook you should be able to continue to the Exploring Queues Notebook, you can ether open it on the explorer or click [Here](./ExploringQueues.ipynb) to open the next notebook.